In [1]:
from pymot import MOTEvaluation
from os import path
from collections import defaultdict

In [2]:
len_seq = 600

In [9]:
def mot(groundtruth, hypotheses):
    evaluator = MOTEvaluation(groundtruth, hypotheses, 0.2)
    evaluator.evaluate()
    print("MOTA: ", evaluator.getMOTA())
    print("MOTP: ", evaluator.getMOTP())
    print(evaluator.getRelativeStatistics())
    print(evaluator.getAbsoluteStatistics())

In [11]:
def load_result():
    data_path = '../dataset/MOT16/train/MOT16-02'
    gt = [[] for i in range(len_seq + 1)]
    result = [[] for i in range(len_seq + 1)]
    with open(path.join(data_path, 'gt', 'gt.txt')) as f:
        for seq, *data in [line.strip().split(',') for line in f.readlines()]:
            gt[int(seq)].append(data)
    
    with open(path.join(data_path, 'MOT16-02.txt')) as f:
        for seq, *data in [line.strip().split(',') for line in f.readlines()]:
            result[int(seq)].append(data)

    return gt, result

In [12]:
def result_to_json(gt, result):
    hypotheses = {}
    groundtruth = {}
    num = 0
    hypotheses['frames'] = []
    hypotheses['class'] = "video"
    hypotheses['filename'] = "../MOT/labs/mot17.idx"
    groundtruth['frames'] = []
    groundtruth['class'] = "video"
    groundtruth['filename'] = "../MOT/labs/mot17.idx"
    for g in gt:
        groundtruth["frames"].append({"timestamp": num})
        groundtruth["frames"][num]["num"] = num
        groundtruth["frames"][num]["class"] = "frame"
        groundtruth["frames"][num]["annotations"] = []
        for obj in g:
            idx, x, y, w, h = obj[0], float(obj[1]), float(obj[2]), float(obj[3]), float(obj[4])
            groundtruth["frames"][num]["annotations"].append({"dco": False, "height": h, "width": w, "id": idx, "y": y, "x":x})
        num += 1
        
    num = 0
    for r in result:
        hypotheses["frames"].append({"timestamp": num})
        hypotheses["frames"][num]["num"] = num
        hypotheses["frames"][num]["class"] = "frame"
        hypotheses["frames"][num]["hypotheses"] = []
        for obj in r:
            idx, x, y, w, h = obj[0], float(obj[1]), float(obj[2]), float(obj[3]), float(obj[4])
            hypotheses["frames"][num]["hypotheses"].append({"height": h, "width": w, "id": idx, "y": y, "x":x})
        num+=1
    return groundtruth, hypotheses

In [13]:
gt, result = load_result()
groundtruth, hypotheses = result_to_json(gt, result)


In [14]:
mot(groundtruth, hypotheses)

MOTA:  0.21676429280992016
MOTP:  0.6889900951615796
{'recoverable mismatch rate': 0.0018497585037508992, 'MOTA': 0.21676429280992016, 'false positive rate': 0.0014387010584729215, 'track precision': 0.978021978021978, 'track recall': 0.6756756756756757, 'mismatch rate': 0.001986777652176892, 'MOTP': 0.6889900951615796, 'non-recoverable mismatch rate': 0.000513821806597472, 'miss rate': 0.77981022847943}
{'lonely ground truth tracks': 24, 'misses': 22765, 'mismatches': 58, 'recoverable mismatches': 54, 'lonely hypothesis tracks': 2, 'non-recoverable mismatches': 15, 'covering hypothesis tracks': 89, 'correspondences': 6428, 'ground truths': 29193, 'false positives': 42, 'ground truth tracks': 74, 'covered ground truth tracks': 50, 'hypothesis tracks': 91, 'total overlap': 4428.8283316986335}
